In [ ]:
%pip install scikit-hts[auto_arima]

In [50]:
from datetime import datetime
from hts import HTSRegressor
from hts.utilities.load_data import load_hierarchical_sine_data
import hts.functions
import calcbench as cb

cb.api_client._rig_for_testing("localhost")

In [61]:
d = cb.standardized_data(
    company_identifiers=["msft", "orcl", "ko"],
    metrics=["GrossProfit", "SIC_Code"],
    all_history=True,
    period_type="annual",
)

In [62]:
one_digit_SIC_codes = pd.to_numeric(
    d["SIC_Code"].iloc[0] // 1000, downcast="integer"
).astype(str)
one_digit_SIC_codes.name = "SIC_Code"

In [63]:
d = d.drop(columns="SIC_Code", level="metric")

In [64]:
d = d.drop(d.index[0]) # drop the 0000 column left over from the SIC_Code

In [65]:
d = d.stack().join(one_digit_SIC_codes).set_index("SIC_Code", append=True)

In [78]:
d.index.levels[0].to_timestamp()

DatetimeIndex(['2007-01-01', '2008-01-01', '2009-01-01', '2010-01-01',
               '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01',
               '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01',
               '2019-01-01', '2020-01-01'],
              dtype='datetime64[ns]', name='period', freq='AS-JAN')

In [74]:
d.index.get_level_values('period').to_timestamp()

DatetimeIndex(['2007-01-01', '2007-01-01', '2007-01-01', '2008-01-01',
               '2008-01-01', '2008-01-01', '2009-01-01', '2009-01-01',
               '2009-01-01', '2010-01-01', '2010-01-01', '2010-01-01',
               '2011-01-01', '2011-01-01', '2011-01-01', '2012-01-01',
               '2012-01-01', '2012-01-01', '2013-01-01', '2013-01-01',
               '2013-01-01', '2014-01-01', '2014-01-01', '2014-01-01',
               '2015-01-01', '2015-01-01', '2015-01-01', '2016-01-01',
               '2016-01-01', '2016-01-01', '2017-01-01', '2017-01-01',
               '2017-01-01', '2018-01-01', '2018-01-01', '2018-01-01',
               '2019-01-01', '2019-01-01', '2019-01-01', '2020-01-01',
               '2020-01-01', '2020-01-01'],
              dtype='datetime64[ns]', name='period', freq=None)

In [80]:
d.index = d.index.set_levels(d.index.levels[0].to_timestamp(), level=0)

In [81]:
d

GrossProfit
period     ticker SIC_Code              
2007-01-01 KO     2         1.845100e+10
           MSFT   7         4.882200e+10
           ORCL   7         1.744900e+10
2008-01-01 KO     2         2.057000e+10
           MSFT   7         4.628200e+10
           ORCL   7         1.845800e+10
2009-01-01 KO     2         1.990200e+10
           MSFT   7         5.008900e+10
           ORCL   7         2.105600e+10
2010-01-01 KO     2         2.242600e+10
           MSFT   7         5.436600e+10
           ORCL   7         2.722400e+10
2011-01-01 KO     2         2.832700e+10
           MSFT   7         5.619300e+10
           ORCL   7         2.912600e+10
2012-01-01 KO     2         2.896400e+10
           MSFT   7         5.746400e+10
           ORCL   7         2.980100e+10
2013-01-01 KO     2         2.843300e+10
           MSFT   7         5.975500e+10
           ORCL   7         3.187500e+10
2014-01-01 KO     2         2.810900e+10
           MSFT   7         6.054200e+10
           ORCL   7         3.069400e+10
2015-01-01 KO     2         2.681200e+10
           MSFT   7         5.837400e+10
           ORCL   7         2.956800e+10
2016-01-01 KO     2         2.539800e+10
           MSFT   7         6.231000e+10
           ORCL   7         3.034000e+10
2017-01-01 KO     2         2.249100e+10
           MSFT   7         7.200700e+10
           ORCL   7         3.132300e+10
2018-01-01 KO     2         2.123300e+10
           MSFT   7         8.293300e+10
           ORCL   7         3.151100e+10
2019-01-01 KO     2         2.264700e+10
           MSFT   7         9.693700e+10
           ORCL   7         3.113000e+10
2020-01-01 KO     2         1.958100e+10
           MSFT   7         1.158560e+11
           ORCL   7         3.262400e+10

In [82]:
d = d.reset_index()

In [88]:
ht, _, _ = hts.functions.get_hierarchichal_df(
    d.reset_index(),
    level_names=["SIC_Code", "ticker"],
    date_colname="period",
    val_colname="GrossProfit",
    hierarchy=[
        ["SIC_Code"],
        ["ticker"],
    ],
)

In [89]:
ht

SIC_Code_ticker,2_KO,7_MSFT,7_ORCL,total,2,7,KO,MSFT,ORCL
period,,,,,,,,,
2007-01-01,1.845100e+10,4.882200e+10,1.744900e+10,8.472200e+10,1.845100e+10,6.627100e+10,1.845100e+10,4.882200e+10,1.744900e+10
2008-01-01,2.057000e+10,4.628200e+10,1.845800e+10,8.531000e+10,2.057000e+10,6.474000e+10,2.057000e+10,4.628200e+10,1.845800e+10
2009-01-01,1.990200e+10,5.008900e+10,2.105600e+10,9.104700e+10,1.990200e+10,7.114500e+10,1.990200e+10,5.008900e+10,2.105600e+10
2010-01-01,2.242600e+10,5.436600e+10,2.722400e+10,1.040160e+11,2.242600e+10,8.159000e+10,2.242600e+10,5.436600e+10,2.722400e+10
2011-01-01,2.832700e+10,5.619300e+10,2.912600e+10,1.136460e+11,2.832700e+10,8.531900e+10,2.832700e+10,5.619300e+10,2.912600e+10
2012-01-01,2.896400e+10,5.746400e+10,2.980100e+10,1.162290e+11,2.896400e+10,8.726500e+10,2.896400e+10,5.746400e+10,2.980100e+10
2013-01-01,2.843300e+10,5.975500e+10,3.187500e+10,1.200630e+11,2.843300e+10,9.163000e+10,2.843300e+10,5.975500e+10,3.187500e+10
2014-01-01,2.810900e+10,6.054200e+10,3.069400e+10,1.193450e+11,2.810900e+10,9.123600e+10,2.810900e+10,6.054200e+10,3.069400e+10
2015-01-01,2.681200e+10,5.837400e+10,2.956800e+10,1.147540e+11,2.681200e+10,8.794200e+10,2.681200e+10,5.837400e+10,2.956800e+10


In [90]:
heirarchy = {"total": ["7", "2"], "7": ["7_MSFT", "7_ORCL"], "2": ["2_KO"]}

In [91]:
clf = HTSRegressor(model="auto_arima", revision_method="OLS", n_jobs=0)

In [93]:
model = clf.fit(ht, heirarchy)

Fitting models: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.68it/s]


In [94]:
predicted_autoarima = model.predict(steps_ahead=4)

Fitting models:   0%|                                                                                                                                                                                           | 0/6 [00:00<?, ?it/s]c:\users\andre\appdata\local\programs\python\python38\lib\site-packages\pmdarima\arima\arima.py:692: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
c:\users\andre\appdata\local\programs\python\python38\lib\site-packages\pmdarima\arima\arima.py:692: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
c:\users\andre\appdata\local\programs\python\python38\lib\site-packages\pmdarima\arima\arima.py:692: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In futur

In [95]:
predicted_autoarima

,total,7,2,7_MSFT,7_ORCL,2_KO
2007-01-01,2.202327e+10,6.405520e+09,1.561775e+10,-3.974017e+09,1.959177e+10,6.405520e+09
2008-01-01,8.744524e+10,4.770040e+10,3.974484e+10,3.703965e+10,2.705189e+09,4.770040e+10
2009-01-01,8.905292e+10,4.782022e+10,4.123270e+10,3.478243e+10,6.450266e+09,4.782022e+10
2010-01-01,9.329778e+10,5.049038e+10,4.280740e+10,3.595006e+10,6.857335e+09,5.049038e+10
2011-01-01,1.072439e+11,5.783283e+10,4.941108e+10,3.786327e+10,1.154780e+10,5.783283e+10
2012-01-01,1.155487e+11,6.256118e+10,5.298749e+10,4.184155e+10,1.114595e+10,6.256118e+10
2013-01-01,1.161362e+11,6.328073e+10,5.285548e+10,4.273767e+10,1.011781e+10,6.328073e+10
2014-01-01,1.206291e+11,6.576226e+10,5.486684e+10,4.361537e+10,1.125147e+10,6.576226e+10
2015-01-01,1.189181e+11,6.532057e+10,5.359751e+10,4.518232e+10,8.415186e+09,6.532057e+10
2016-01-01,1.148886e+11,6.305722e+10,5.183133e+10,4.354780e+10,8.283530e+09,6.305722e+10
